If the clingo application is build with Python support, clingo will also be able to execute Python code embedded in logic programs. Functions defined in a Python script block are callable during the instantiation process using @-syntax. The default grounding/solving process can be customized if a main function is provided.

Note that gringo's precomputed terms (terms without variables and interpreted functions), called symbols in the following, are wrapped in the Symbol class. Furthermore, strings, numbers, and tuples can be passed wherever a symbol is expected - they are automatically converted into a Symbol object. Functions called during the grounding process from the logic program must either return a symbol or a sequence of symbols. If a sequence is returned, the corresponding @-term is successively substituted by the values in the sequence.

# Using Clingo within Python

In [5]:
from clingo.symbol import Number
from clingo.control import Control

class Context:
    def inc(self, x):
        # numerical values will be converted to a Symbol Object
        return Number(x.number + 1)
    def seq(self, x, y):
        return [x, y]

def on_model(m):
    print (m)

ctl = Control()

ctl.add("base", [], """\
p(@inc(10)).
q(@seq(1,2)).
""")

ctl.ground([("base", [])], context=Context())

print(ctl.solve(on_model=on_model))

p(11) q(1) q(2)
SAT


In [6]:
from clingo.symbol import Number
from clingo.control import Control

class Context:
    def inc(self, x):
        return Number(x.number + 1)
    def seq(self, x, y):
        return [x, y]

def on_model(m):
    print (m)

ctl = Control()

ctl.add("base", [], """\
p(@inc(10)).
q(@seq(1,2)).
""")

ctl.ground([("base", [])], context=Context())

with ctl.solve(yield_=True) as handle:
    for m in handle: print("Answer: {}".format(m))
    handle.get()

Answer: p(11) q(1) q(2)


# Using Python within Clingo

In [2]:
%%file example.lp

#script (python)

from clingo.symbol import Number

class Context:
    def inc(self, x):
        return Number(x.number)

    def seq(self, x, y):
        return [x, y]

def main(prg):
    prg.ground([("base", [])], context=Context())
    prg.solve()

#end.

p(@inc(10)).
q(@seq(1,2)).

Overwriting example.lp


In [3]:
!clingo example.lp -n0

clingo version 5.5.1
Reading from example.lp
Solving...
Answer: 1
p(10) q(1) q(2)
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.109s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.109s
